# Assignment 3: MapReduce & Web Graph

## General guidelines

This notebook contains considerable amount of code to help you complete this assignment. Your task is to implement any missing parts of the code and answer any questions (if exist) within this notebook. This will require understanding the existing code, may require reading about packages being used, reading additional resources, and maybe even going over your notes from class :😱

**Evaluation and auto-grading**: Your submissions will be evaluated using both automatic and manual grading. Code parts for implementation are marked with a comment `# YOUR CODE HERE`, and usually followed by cell(s) containing automatic tests that evaluate the correctness of your answer. Staff will allow your notebook to **execute from start to finish for no more than 90 seconds**, then manually assess your submission. Any automatic tests that did not run due to your notebook timing out **will automatically receive 0 points**. The execution time excludes initial data download, which will already exist in the testing environment. The staff reserves the right to **modify any grade provided by the auto-grader** as well as to **execute additional tests not provided to you**. It is also important to note that **auto-graded cells only result in full or no credit**. In other words, you must pass all tests implemented in a test cell in order to get the credit for it, and passing some, but not all, of the tests in a test cell will not earn you any points for that cell.

**Submission**: Upload a zip file **with your ID as the file name** containing the colab notebook and the GCP notebook to the assignment on Moodle. The names of the files should be  in the format:  012345678_colab.ipynb and 012345678_gcp.ipynb.

# Tasks

In this assignment, we are going to use MapReduce in pySpark to scale up index creation. **The assignment consists of two parts, one in Google Colab and one in GCP**. In the first part (Colab), we will learn how to use MapReduce in PySpark, create an inverted index, and calculate PageRank. In the second part, we will use GCP to scale up index creation for the entire English Wikipedia.

You will learn how to work with PySpark, write MapReduce code that executes on a cluster of machines, and experience the power of parallel processing. **All tasks must be completed through the use of PySpark's MapReduce**.

To save you (and us) time, both in this assignment and in the final project, we are providing you with preprocessed files for the entire English wikipedia dump. Preprocessing was done, similar to Assignment #2, to extract article information from the XML, then remove the WikiMedia markup. The complete preprocessing code for one wiki dump can be seen [here](https://colab.research.google.com/drive/1xXc0YPU4BVd8CAjVNgICimW_d-Z7wryu?usp=sharing).


**By the end of this assignment, you will have:**

In Colab (65 Points)

1. (20 Points) Processed articles' body to produce posting lists (using PySpark and MapReduce).
2. (5 Points) Calculated document frequency (DF) in a  distributed fashion.
3. (20 Points) Partitioned the posting lists and wrote them to disk in parallel.
4. (20 Points) Constructed the links graph and calculated PageRank on it.
All of these will be based on a small subset of articles from one wiki dump file.

In GCP (35 Points):

1. (10 Points) Successfully started a Dataproc cluster, setup cloud storage bucket, and properly initialized cluster nodes.
2. (10 Points) Created an inverted index for all of English Wikipedia.
3. (10 Points) Successfully computed PageRank for the entire English Wikipedia.
4. (5 Points) Reporting: size of the data, size of index, time it took to build the index, time it took to compute PageRank, and the $ amount of credits it consumed.

Bonus (10 points) if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour.







# Setup

## General imports

The `inverted_index_colab` import requires the `inverted_index_colab.py` file. You should upload the file and then run this cell.

In [ ]:
# !pip install google-cloud-storage

ERROR: Operation cancelled by user


In [1]:
import sys
# from collections import Counter, OrderedDict
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Installing, importing, and initializing PySpark


The following code installs PySpark and its dependencies in Colab. In addition, we install GraphFrames, which is a PySpark package for dealing with graphs in a distributed fashion. Colab notebooks run on a single machine so we will work in local mode, i.e. there is no cluster of machines and both the master and worker processes run on a single machine. This will help us debug and iron out the code we will use in the second half of this assignment on an actual cluster mode in GCP.

The installation in the next cell should take about 1 minute in a fresh environment. Don't worry about going  over the 90 seconds limit for the assignment because in our testing enviroment all of these requirements will already be met.

In [2]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u432-ga~us1-0ubuntu2~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u432-ga~us1-0ub

In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [5]:
spark

## Copy some wiki data

As mentioned above, all wiki dumps were preprocessed and placed in a shared google storage bucket. To access the files in Colab, you will first have to authenticate with google storage, and then copy the data to your local environment.

**IMPORTANT NOTE**

Before you start working with the wiki data, you first have to go over the
"Working with GCP.pdf" Provided to you under the gcp folder in the same zip file as this folder. In that pdf you will redeem your credits for the GCP and create your instance. You have to do this procedure because the files will be mounted to you through GCP process. Important: Ensure you have completed the **Google Storage** section of the PDF document before continuing with the next steps.
Notice that you have to change the project_id and data_bucket_name variables in the second cell below.

In GCP, the storage will already be mounted on the cluster machines and we will show you how to access it.


In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 100

def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
  signal.signal(signal.SIGALRM, handler)
  signal.alarm(AUTH_TIMEOUT)
  auth.authenticate_user()
  signal.alarm(0)
except:
  pass

Run this cell and follow the authorization link that appears. The link will take you to Google Cloud Platform (GCP) where you'll need to:

Sign in to your Google account
Copy the verification code provided
Return here to paste the code when prompted

In [7]:
import os
from pathlib import Path
from google.colab import auth
from google.cloud import storage

# Alternative authentication method
!gcloud auth login

# Set your project and bucket info
project_id = 'ir-hw3-444616'
data_bucket_name = 'hw3ir322'

# Configure project
!gcloud config set project {project_id}

# Create directory and copy file directly
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
    !mkdir -p wikidumps
    !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.

# Creating local folders

In [8]:
import os
import shutil

!mkdir -p text_index
!mkdir -p title_index
!mkdir -p anchor_index

def move_bin_files(folder_name):
    """
    Moves all files ending with '.bin' in the current directory
    to the specified folder.

    Args:
        folder_name (str): The name of the destination folder.
    """
    for filename in os.listdir():
        if filename.endswith(".bin") or filename.endswith(".pkl"):
            source_path = os.path.abspath(filename)
            destination_path = os.path.join(folder_name, filename)

            try:
                shutil.move(source_path, destination_path)
            except Exception as e:
                print(f"Error moving '{filename}': {e}")


# Processing wikipedia

Now that we completed the setup and have some data in our local environment, we are ready to process it using PySpark.

## A 2-minute intro to PySpark

One of the fundamental data structures in spark is a **Resillient Distributed Dataset (RDD)**. It is an immutable distributed collections of objects that are partitioned across machines, and thus making parallel processing natural and easy. RDD's support two types of operations: *transformations* (e.g. selecting a subset of fields from each element or projecting each element from A->B), and *actions* (e.g. save to disk or `collect` elements into the master node's memory). Watch out for `collect` operations that send too much data to the master node, it will crash without a trace because of running out of memory.

Spark employs **lazy-evaluation** (like many of us...), meaning that it accumulates things we ask it to do into an execution plan, but doesn't do any calculation until a result is actually needed. When a result is needed? when we save something to disk, ask spark to show the data, or use other actions. But make no mistakes, Spark is not a slacker. It employs lazy-evaluation because it allows optimizations that wouldn't be possible otherwise and it often saves users from unnecessary data reads, tranfers across machines, and other costly operations. As we will see in a bit, the RDD data structure lends itself to simple and efficient MapReduce operations.

Let's look at our data before transforming it to RDD.

In [9]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

# Making pairs for title, text and anchor text

In [10]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text", "id").rdd

## Word counts

### Term frequency

**YOUR TASK (10 POINTS):** Complete the implementation of `word_count`, which calculates the term-frequency (tf) for each word in a single document.  map function. The functions gets the text and the wiki id, calculates the tf and returns list of tuples in the form of (token, (wiki_id, tf)).

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  # tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # # YOUR CODE HERE
  # res_list = []
  # for token in tokens:
  #   if token not in all_stopwords:
  #     if (token, (id, tokens.count(token))) not in res_list:
  #       res_list.append((token, (id, tokens.count(token))))
  # return res_list

  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  token_counts = Counter(tokens)  # Calculate term frequencies efficiently
  res_list = [
        (token, (id, count))
        for token, count in token_counts.items()
        if token not in all_stopwords
    ]
  return res_list
  # raise NotImplementedError()

In [12]:
def tokenize(text):
  english_stopwords = frozenset(stopwords.words('english'))
  corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
  RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

  all_stopwords = english_stopwords.union(corpus_stopwords)
  stemmer = PorterStemmer()
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  stemmed = [stemmer.stem(token) for token in tokens if token not in all_stopwords]
  return stemmed

In [13]:
def stemmed_word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = tokenize(text)
  token_counts = Counter(tokens)  # Calculate term frequencies efficiently
  res_list = [
        (token, (id, count))
        for token, count in token_counts.items()
        if token not in all_stopwords
    ]
  return res_list

In [14]:
text_doc_lengths_dict = {}
title_doc_lengths_dict = {}

def len_tokens(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  return (id, len(tokens))

# Apply the function to your RDD
text_len_pairs = doc_text_pairs.map(lambda x: len_tokens(x[0], x[1]))
text_doc_lengths_dict = text_len_pairs.collectAsMap()

title_len_pairs = doc_title_pairs.map(lambda x: len_tokens(x[0], x[1]))
title_doc_lengths_dict = title_len_pairs.collectAsMap()

In [15]:
print(text_doc_lengths_dict)
print(title_doc_lengths_dict)

{12: 4179, 25: 4270, 39: 1868, 290: 1034, 303: 8281, 305: 4386, 307: 8137, 308: 6177, 309: 1043, 316: 4632, 324: 4456, 330: 251, 332: 237, 334: 694, 336: 3888, 339: 4274, 340: 305, 344: 818, 358: 6744, 359: 2613, 569: 4274, 572: 639, 573: 4985, 579: 442, 580: 465, 586: 3451, 590: 196, 593: 3709, 594: 10157, 595: 5471, 597: 2332, 599: 3238, 600: 5228, 612: 752, 615: 918, 620: 3823, 621: 7070, 624: 7261, 627: 4850, 628: 2543, 630: 376, 632: 637, 633: 3059, 634: 2816, 639: 4067, 640: 1970, 642: 147, 643: 614, 649: 565, 651: 1137, 653: 2945, 655: 2226, 656: 3128, 657: 4628, 659: 1027, 661: 132, 662: 6816, 663: 4536, 664: 2854, 665: 1543, 666: 9421, 670: 2811, 673: 985, 674: 3756, 675: 408, 676: 3377, 677: 2160, 678: 277, 679: 583, 680: 1790, 681: 1056, 682: 1475, 683: 418, 689: 4584, 690: 2961, 691: 3825, 694: 39, 698: 5089, 700: 7464, 701: 6012, 704: 1125, 705: 702, 706: 1840, 708: 233, 709: 1290, 710: 1500, 711: 2773, 713: 1682, 717: 6386, 728: 648, 734: 1645, 736: 7304, 737: 8274, 738: 

In [16]:
stemmed_text_word_counts = doc_text_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))
stemmed_title_word_counts = doc_text_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))

new_pairs = doc_anchor_pairs.flatMap(lambda pair: [(p[1], p[0]) for p in pair[0]])
stemmed_anchor_word_counts = new_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))

In [17]:
# test for word count
text = "This is a test to check the words count function actually count words"
id = 0
word_count_results = word_count(text, id)
assert word_count_results[0][0] == 'test'
assert len(word_count_results) == 6
assert word_count_results[2][1][1] == word_count_results[2][1][1]
assert type(word_count_results[0][0]) == str
assert type(word_count_results[0][1]) == tuple


In [18]:
stemmed_word_count_res = stemmed_word_count(text, id)
print("stemmed:", stemmed_word_count_res)
print("not stemmed:", word_count_results)
# assert stemmed_word_count_res[0][0] == 'test'
# assert len(stemmed_word_count_res)

stemmed: [('test', (0, 1)), ('check', (0, 1)), ('word', (0, 2)), ('count', (0, 2)), ('function', (0, 1)), ('actual', (0, 1))]
not stemmed: [('test', (0, 1)), ('check', (0, 1)), ('words', (0, 2)), ('count', (0, 2)), ('function', (0, 1)), ('actually', (0, 1))]


PySpark has a `map` trasformation that, just like we learned in class about Map in MapReduce, executes a given function on each element of an  RDD. It also has a `flatMap()` transformation that executes a given function on each element of an RDD, but afterwards flattens the results to create one big "list" instead multiple nested lists. For example, `flatMap` on the above `word_count` function will concatenate the tokens from multiple document into one big list rather than keeping separate list of tokens for each document like a regular `map` would produce.

In [ ]:
#word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))

After `flatMap` is executed, the counts for a given token are likely to be scattered accross multiple machines/workers in the cluster. In order to create the posting lists for each token, we need a reduce operation that collects them into one list and sorts them by the document id (wiki_id).

Spark has several ways of reducing or aggergating values, but we will use `groupByKey`, which like Reduce in MapReduce, collects all the values for a given key into an iterable list of the values associated with that key. The result of `groupByKey` is an RDD with (key, value*s*) pairs. Then, we can call `map` to execute a function on each pair in that RDD or call `mapValues` to execute a function only on the values iterable.

**YOUR TASK (5 POINTS):** Complete the implementation of `reduce_word_counts`, which operates on the pairs returned by `word_count`. The functions gets a list of values (unsorted posting list) and sorts it.

In [19]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  # sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted(unsorted_pl, key=lambda x: x[0])
  # raise NotImplementedError()

In [ ]:
#postings = word_counts.groupByKey().mapValues(reduce_word_counts)

# postings for text, title and anchor

In [20]:
text_postings = stemmed_text_word_counts.groupByKey().mapValues(reduce_word_counts)
title_postings = stemmed_title_word_counts.groupByKey().mapValues(reduce_word_counts)
anchor_postings = stemmed_anchor_word_counts.groupByKey().mapValues(reduce_word_counts)

def new_count(posting):
    counts = Counter(p[0] for p in posting)
    return list(counts.items())

# Apply the group function to the values of each key in posting_anchor
anchor_postings = anchor_postings.map(lambda x: (x[0], new_count(x[1])))

In [ ]:
# DO NOT RUN !!!!!!!

## test reduce_word_count
# token, posting_list = postings.take(1)[0]
# assert type(token) == str
# assert type(posting_list) == list
# doc_id, tf = zip(*posting_list)
# assert type(doc_id[0]) == int
# assert type(tf[0]) == int
# assert np.diff(doc_id).min() > 0
# assert np.min(tf) > 0
# assert postings.count() == 153014

### Document frequency and filteration

Next, we will filter out rare words, words that appear in 10 or fewer documents (when working on the entire corpus, we will increase this threshold to a minimum of 50 documents).

In [ ]:
# DO NOT RUN !!!!!!!
# postings_filtered = postings.filter(lambda x: len(x[1])>10)

# filter postings

In [21]:
text_postings_filtered = text_postings.filter(lambda x: len(x[1])>10)
title_postings_filtered = title_postings.filter(lambda x: len(x[1])>10)
anchor_postings_filtered = anchor_postings.filter(lambda x: len(x[1])>10)

Now, we would like to calculate the df for each word.

**YOUR TASK (5 POINTS):** Complete the implementation of `calculate_df` that takes a posting list RDD and returns an RDD where each element is a (token, df) pair.

In [22]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  res_rdd = postings.map(lambda x: (x[0], len(x[1])))
  return res_rdd
  # raise NotImplementedError()

In [ ]:
# DO NOT RUN !!!!!!!

# # global statistics
# w2df = calculate_df(postings_filtered)

# # test calculate_df types
# token, df = w2df.take(1)[0]
# assert type(token) == str
# assert type(df) == int
# # test min/max df values
# # collectAsMap collects the results to the master node's memory as a dictionary.
# # we know it's not so big so this is okay.
# w2df_dict = w2df.collectAsMap()
# assert np.min(list(w2df_dict.values())) == 11
# assert np.max(list(w2df_dict.values())) == 819
# # test select words
# assert w2df_dict['first'] == 805
# assert w2df_dict['many'] == 670
# assert w2df_dict['used'] == 648

# w2df

In [23]:
text_w2df = calculate_df(text_postings_filtered)
title_w2df = calculate_df(title_postings_filtered)
anchor_w2df = calculate_df(anchor_postings_filtered)

text_w2df_dict = text_w2df.collectAsMap()
title_w2df_dict = title_w2df.collectAsMap()
anchor_w2df_dict = anchor_w2df.collectAsMap()

## Partitioning and writing the index

Now that we have the postings information we need to write it to disk. We would like to partition the tokens and posting lists into buckets so when we get a query, we know exactly in which bucket to look for its posting list. This can also help parallelize retrieval from the index if we have multiple machines answering queries.

Here, you are asked to do this partitioning/bucketing through another round of map and reduce operations, where the last step should be to write out each bucket into a separate inverted index on disk. Use the staff-provided function of  `write_a_posting_list` implemented in inverted_index_colab.py for the InvertedIndex class. Eventually, this partitioning and writing process needs to return information about where each posting list is located (file name and offset) for a token.

**YOUR TASK (20 POINTS):** Complete the implementation of `partition_postings_and_write` that partitions the posting list, writes out each bucket, and returns information about the location on disk of each posting list.

In [25]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # YOUR CODE HERE
  bucket_rdd = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()

  def write_bucket(b_w_pl):
      bucket_id, word_posting_pairs = b_w_pl
      return InvertedIndex.write_a_posting_list((bucket_id, list(word_posting_pairs)))

  posting_locs_list = bucket_rdd.map(write_bucket)

  return posting_locs_list

The next cell uses the `collect` operator to aggregate the posting location information from the multiple buckets/indices into a single dictionary. We know in this case that the posting locations info is relatively small so this is not going to crash our master node.

In [ ]:
# DO NOT RUN !!!!!!!

# test partitioning for the different buckets (10 points)
# posting_locs_list = partition_postings_and_write(postings_filtered).collect()
# assert len(posting_locs_list) == 124
# assert len(list(Path('.').glob('*.bin'))) == 124
# pl_lens = [len(pl) for pl in posting_locs_list]
# assert np.max(pl_lens) == 141
# assert np.min(pl_lens) == 91
# assert int(np.median(pl_lens)) == 116

# partition posting lists and write out

In [26]:
text_posting_locs_list = partition_postings_and_write(text_postings_filtered).collect()

In [ ]:
# DO NOT RUN !!!!!!!

# merge the posting locations into a single dict and run more tests (5 points)
# super_posting_locs = defaultdict(list)
# for posting_loc in posting_locs_list:
#   for k, v in posting_loc.items():
#     super_posting_locs[k].extend(v)

# assert super_posting_locs['try'][0][0][:3] == '105'
# assert super_posting_locs['political'][0][0][:2] == '83'
# assert 'hi' not in super_posting_locs
# assert len(super_posting_locs) == 14524

# text super postings locs

In [27]:
text_super_posting_locs = defaultdict(list)
for posting_loc in text_posting_locs_list:
  for k, v in posting_loc.items():
    text_super_posting_locs[k].extend(v)

Putting everything together (posting locations, df) and test that the resulting index is correct.

In [ ]:
# DO NOT RUN !!!!!!!

# # Create inverted index instance
# inverted = InvertedIndex()
# # Adding the posting locations dictionary to the inverted index
# inverted.posting_locs = super_posting_locs
# # Add the token - df dictionary to the inverted index
# inverted.df = w2df_dict
# # write the global stats out
# inverted.write_index('.', 'index')

NameError: name 'super_posting_locs' is not defined

# putting it together for text

In [28]:
# Create inverted index instance
text_inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
text_inverted.posting_locs = text_super_posting_locs
# Add the token - df dictionary to the inverted index
text_inverted.df = text_w2df_dict
# write the global stats out
# text_inverted.write_index('.', 'index')
text_inverted.write_index('.', 'text')

move_bin_files('text_index')

# putting it together for title

In [29]:
title_posting_locs_list = partition_postings_and_write(title_postings_filtered).collect()

title_super_posting_locs = defaultdict(list)
for posting_loc in title_posting_locs_list:
  for k, v in posting_loc.items():
    title_super_posting_locs[k].extend(v)

# Create inverted index instance
title_inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
title_inverted.posting_locs = title_super_posting_locs
# Add the token - df dictionary to the inverted index
title_inverted.df = title_w2df_dict
# write the global stats out
title_inverted.write_index('.', 'title')

move_bin_files('title_index')

# putting it together for anchor

In [30]:
anchor_posting_locs_list = partition_postings_and_write(anchor_postings_filtered).collect()

anchor_super_posting_locs = defaultdict(list)
for posting_loc in anchor_posting_locs_list:
  for k, v in posting_loc.items():
    anchor_super_posting_locs[k].extend(v)

# Create inverted index instance
anchor_inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
anchor_inverted.posting_locs = anchor_super_posting_locs
# Add the token - df dictionary to the inverted index
anchor_inverted.df = anchor_w2df_dict
# write the global stats out
anchor_inverted.write_index('.', 'anchor')

move_bin_files('anchor_index')

In [31]:
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [ ]:
# # DO NOT RUN !!!!!!!

# ## test the final index
# pl = read_posting_list(inverted, 'clarity')
# assert pl[-3] == (2204, 2)
# assert len(pl) == 18

NameError: name 'inverted' is not defined

## PageRank

In this section, your task is to compute PageRank for wiki articles using the anchor text we extracted from the MediaWiki markdown. Let's select the 'id' and 'anchor_text' fields into a new RDD:

In [ ]:
pages_links = spark.read.parquet(path).limit(1000).select("id", "anchor_text").rdd

**YOUR TASK (20 POINTS):** Complete the implementation of `generate_graph`, which generates the graph of internal wikipedia links (wiki articles pointing to other wiki articles). Use PySpark to compute the vertices (pages) and edges (links) of the graph. Multiple links from page A to page B need to be represented by a single edge (edges are not weighted). You may use the built-in function `distinct` of the RDD object to identify distinct elements, but be mindful of its impact on computation time.

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  # YOUR CODE HERE
  # raise NotImplementedError()

  # edges = pages.flatMap(lambda x: [(x[0], anchor) for anchor in x[1]])
  # edges = edges.mapValues(lambda x: (x[0],))
  edges = pages.flatMap(lambda x: [(x[0], anchor[0]) for anchor in x[1]])
  edges = edges.distinct()

  # vertices = edges.flatMap(lambda x:  [(x[0],), (x[1],)])
  # vertices = edges.flatMap(lambda x: [x[0], x[1][0]]).distinct()
  vertices = edges.flatMap(lambda x: [x[0], x[1]]).distinct()
  vertices = vertices.map(lambda x: Row(id=x))

  # vertices = vertices.map(lambda x: Row(id=x))
  # vertices = vertices.distinct()




  return edges, vertices

In [ ]:
# test graph construction efficiency (10 points)
t_start = time()
# construct the graph for a small sample of (1000) pages
edges, vertices = generate_graph(pages_links)
# time the actual execution
v_cnt, e_cnt = vertices.count(), edges.count()
graph_const_time = time() - t_start
# test that the implementation is efficient (<20 seconds)
assert graph_const_time < 20

In [ ]:
# test that we get the right number of vertices (5 points)
print(v_cnt)
assert v_cnt == 114966

114966


In [ ]:
# test that we get the right number of edges (5 points)
print(e_cnt)
assert e_cnt == 183727

183727


Let's compute PageRank in a distributed fashion (using PySpark's GraphFrame package) and show the top 20 pages.

In [ ]:
edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv('pr', compression="gzip")
pr.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+--------+------------------+
|      id|          pagerank|
+--------+------------------+
|  606848|1.9768910954777374|
|43432956|1.7183234055660317|
|37404646| 1.561180019985396|
|  148363| 1.531189053212204|
|     308| 1.523973241896932|
|     783|1.5205983727677175|
|   39235|1.5113438223085562|
|   30680| 1.506487061855029|
| 3434750|1.5042432651671351|
|   48518|1.5032093881505733|
| 5843419| 1.475564533072746|
|   25458|1.4651925575346503|
|     854| 1.433614708798739|
|   14532|1.4318560075868727|
|   11887| 1.422942493084949|
|   32927|1.4195131540605397|
| 8034453| 1.414325748041313|
|    1686|1.4005429901986137|
|   25507|1.3695924013144563|
|   61371|1.3682656561066355|
+--------+------------------+
only showing top 20 rows



**YOUR TASK (5 POINTS)**: Complete the implmentation of `get_top_pr_page_titles` to return the titles a few of the top pages as ranked by PageRank and printed above. To get the title for each page id listed above, navigate to the url https://en.wikipedia.org/?curid=wiki_id_here by replacing the `wiki_id_here` part with the page id. For example, if you navigate to https://en.wikipedia.org/?curid=25507 for the page with id 25507 you'd see that its title is 'Roman Empire'.

In [ ]:
def get_top_pr_page_titles():
  ''' Returns the title of the first, fourth, and fifth pages as ranked about
      by PageRank.
  Returns:
  --------
    list of three strings.
  '''
  # YOUR CODE HERE

  # raise NotImplementedError()

In [ ]:
assert [_hash(x.lower()) for x in get_top_pr_page_titles()] == \
  ['52bb7dbc02', '3e920ea944', '3d007b19ee']

Do the top ranked pages make sense to you? Why do you think these pages received such a high scores from PageRank?

# GCP

The second part of the assignment involves running index creation in GCP and reporting results in this notebook. Follow [this](https://docs.google.com/document/d/1HTZVDMQ1gyeq8dlyzmomOkxjSo1Td76PsCGlFkQohps/edit?usp=sharing) step-by-step guide in order to set up your cluster environment on GCP so you can execute pyspark code on a cluster of machines.

The code to the GCP notebook is in the moodle under assignment 3. You will have to copy the code you completed here in the relevant sections.

In the current notebook we used MapReduce to create the index for only one wiki dump. Using GCP, we will be able to create the index on the entire english wikipedia. The GCP part of this assignment will be the first time we will work with the entire corpus. You will need to follow the notebook, complete the code as you did here (the same code you completed in this notebook can be used in the GCP notebook).
At the end of the GCP assignment you will have questions that you will need to answer - this are not the tests we used so far, so you should follow the instructions in the notebook.

# BM25 class

In [ ]:
# # Modify BM25 class to overcome DL - DL: fetches the document length of each document.

# import math
# from itertools import chain
# import time
# # When preprocessing the data have a dictionary of document length for each document saved in a variable called `DL`.
# class BM25_from_index:
#     """
#     Best Match 25.
#     ----------
#     k1 : float, default 1.5

#     b : float, default 0.75

#     index: inverted index
#     """

#     def __init__(self,index,DL,k1=1.5, b=0.75):
#         self.b = b
#         self.k1 = k1
#         self.index = index
#         self.N = len(DL)
#         # self.AVGDL = sum(DL.values())/self.N
#         self.AVGDL = sum(list(DL.values())) / self.N
#         self.words, self.pls = zip(*self.index.posting_lists_iter())

#     def calc_idf(self,list_of_tokens):
#         """
#         This function calculate the idf values according to the BM25 idf formula for each term in the query.

#         Parameters:
#         -----------
#         query: list of token representing the query. For example: ['look', 'blue', 'sky']

#         Returns:
#         -----------
#         idf: dictionary of idf scores. As follows:
#                                                     key: term
#                                                     value: bm25 idf score
#         """
#         idf = {}
#         for term in list_of_tokens:
#             if term in self.index.df.keys():
#                 n_ti = self.index.df[term]
#                 idf[term] = math.log(1 + (self.N - n_ti + 0.5) / (n_ti + 0.5))
#             else:
#                 pass
#         return idf


#     def search(self, queries,N=3):
#         """
#         This function calculate the bm25 score for given query and document.
#         We need to check only documents which are 'candidates' for a given query.
#         This function return a dictionary of scores as the following:
#                                                                     key: query_id
#                                                                     value: a ranked list of pairs (doc_id, score) in the length of N.

#         Parameters:
#         -----------
#         query: list of token representing the query. For example: ['look', 'blue', 'sky']
#         doc_id: integer, document id.

#         Returns:
#         -----------
#         score: float, bm25 score.
#         """
#         # YOUR CODE HERE
#         query_scores = {} # init dict

#         for query_id, query_tokens in queries.items():
#             self.idf = self.calc_idf(query_tokens) # calc idf for current query

#             if not self.idf:
#                 query_scores[query_id] = []  # no relevant documents
#                 continue

#             # getting candidtaes
#             candidate_docs = set()
#             for term in query_tokens:
#                 if term in self.index.df:
#                     posting_list = self.pls[self.words.index(term)]
#                     candidate_docs.update([doc_id for doc_id, _ in posting_list])

#             # calc common terms' data
#             common_terms_data = {}
#             for term in query_tokens:
#                 if term in self.index.df:
#                     posting_list = self.pls[self.words.index(term)]
#                     common_terms_data[term] = dict(posting_list)

#             # calc scores for candidates
#             doc_scores = {}
#             for doc_id in candidate_docs:
#                 score = 0.0
#                 doc_len = DL[str(doc_id)]

#                 for term in query_tokens:
#                     if term in common_terms_data and doc_id in common_terms_data[term]:
#                         freq = common_terms_data[term][doc_id]
#                         numerator = self.idf[term] * freq * (self.k1 + 1)
#                         denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
#                         score += (numerator / denominator)

#                 doc_scores[doc_id] = score

#             sorted_scores = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
#             top_n_docs = sorted_scores[:N]

#             query_scores[query_id] = top_n_docs

#         return query_scores

#     def _score(self, query, doc_id):
#         """
#         This function calculate the bm25 score for given query and document.

#         Parameters:
#         -----------
#         query: list of token representing the query. For example: ['look', 'blue', 'sky']
#         doc_id: integer, document id.

#         Returns:
#         -----------
#         score: float, bm25 score.
#         """
#         score = 0.0
#         doc_len = DL[str(doc_id)]

#         for term in query:
#             if term in self.index.term_total.keys():
#                 term_frequencies = dict(self.pls[self.words.index(term)])
#                 if doc_id in term_frequencies.keys():
#                     freq = term_frequencies[doc_id]
#                     numerator = self.idf[term] * freq * (self.k1 + 1)
#                     denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
#                     score += (numerator / denominator)
#         return score

# read indices

In [32]:
idx_text = InvertedIndex.read_index('text_index', 'text')
idx_title = InvertedIndex.read_index('title_index', 'title')
idx_anchor = InvertedIndex.read_index('anchor_index', 'anchor')

In [33]:
# bm25_title = BM25_from_index(idx_title, title_doc_lengths_dict)
print(type(title_doc_lengths_dict))

<class 'dict'>


In [34]:
print(title_doc_lengths_dict.items())
print(text_doc_lengths_dict.items())

dict_items([(12, 1), (25, 1), (39, 1), (290, 0), (303, 1), (305, 1), (307, 2), (308, 1), (309, 2), (316, 5), (324, 2), (330, 1), (332, 2), (334, 3), (336, 1), (339, 2), (340, 2), (344, 2), (358, 1), (359, 4), (569, 1), (572, 2), (573, 1), (579, 1), (580, 1), (586, 1), (590, 2), (593, 1), (594, 1), (595, 2), (597, 2), (599, 2), (600, 1), (612, 2), (615, 3), (620, 2), (621, 1), (624, 1), (627, 1), (628, 2), (630, 1), (632, 2), (633, 1), (634, 2), (639, 1), (640, 4), (642, 2), (643, 2), (649, 1), (651, 2), (653, 2), (655, 1), (656, 1), (657, 1), (659, 4), (661, 2), (662, 1), (663, 1), (664, 1), (665, 2), (666, 2), (670, 1), (673, 2), (674, 1), (675, 2), (676, 2), (677, 1), (678, 1), (679, 2), (680, 1), (681, 1), (682, 1), (683, 1), (689, 1), (690, 1), (691, 2), (694, 3), (698, 2), (700, 2), (701, 1), (704, 2), (705, 2), (706, 2), (708, 2), (709, 3), (710, 3), (711, 3), (713, 2), (717, 1), (728, 2), (734, 1), (736, 2), (737, 1), (738, 1), (740, 1), (742, 2), (746, 1), (748, 2), (751, 1), (

In [35]:
print(len(text_doc_lengths_dict))

1000


## Our implementation for BM25 from index

In [36]:
import math
import builtins
from itertools import chain
import time
from inverted_index_colab import *
# When preprocessing the data have a dictionary of document length for each document saved in a variable called `DL`.

class BM25:
    """
    Best Match 25.
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    index: inverted index
    """

    def __init__(self,index,DL,k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.index = index
        self.N = len(DL)
        # self.AVGDL = sum(DL.values())/self.N
        self.AVGDL = builtins.sum(DL.values()) / self.N
        # self.AVGDL = math.sum(DL.values()) / self.N
        self.words, self.pls = zip(*self.index.posting_lists_iter_inside_dir())


    def print_index():
      print(self.index)

In [ ]:
import builtins
val_sum = builtins.sum(text_doc_lengths_dict.values())
print(val_sum)

2201829


In [37]:
# bm25_text = BM25(idx_text, text_doc_lengths_dict)
# bm25_text.print_index()

NameError: name 'base_dir' is not defined

In [47]:
text_pkl = idx_text.read_index("/content/text_index","text")
print(text_pkl.posting_locs)

defaultdict(<class 'list'>, {'polit': [('74_000.bin', 0)], 'revolutionari': [('74_000.bin', 1902)], 'cynic': [('74_000.bin', 2466)], 'gave': [('74_000.bin', 2562)], 'fertil': [('74_000.bin', 4350)], 'lead': [('74_000.bin', 4740)], 'suppress': [('74_000.bin', 7434)], 'lenin': [('74_000.bin', 7974)], 'feminist': [('74_000.bin', 8088)], 'affin': [('74_000.bin', 8268)], 'primari': [('74_000.bin', 8496)], 'combin': [('74_000.bin', 10050)], 'tool': [('74_000.bin', 12204)], 'increasingli': [('74_000.bin', 12996)], 'grant': [('74_000.bin', 13824)], 'rais': [('74_000.bin', 15084)], 'lessen': [('74_000.bin', 16524)], 'outlin': [('74_000.bin', 16602)], 'hugh': [('74_000.bin', 17142)], 'plant': [('74_000.bin', 17652)], '9th': [('74_000.bin', 18744)], 'planter': [('74_000.bin', 19218)], 'passag': [('74_000.bin', 19308)], 'influx': [('74_000.bin', 19980)], 'reptil': [('74_000.bin', 20154)], 'hall': [('74_000.bin', 20274)], 'visitor': [('74_000.bin', 21318)], 'championship': [('74_000.bin', 21786)], 